# 介紹
1. 2020/6/5 測試可正常爬資料，

如果遇到無法爬資料可能是104、1111有改版導致BeautifulSoup無法找到calss、tag、attr，

**發生無法爬取從以下修改def**
* 104人力銀行
    * find_title_104 爬資料
    * selenium_get_Code_104 取得最大頁面數
* 1111人力銀行
    * find_title_1111 爬資料
    * find_data_1111 指定class、tag、id 並丟給find_title_1111
    * selenium_get_Code_1111 取得最大頁面數
    

2. 只爬取104，1111因為比較多人使用。
3. 爬取大量資料並使用Pandas處理資料及過濾資料，找有興趣的相關工作。 


104爬取資料有『日期、工作名稱、公司名稱、公司地址、薪資、工作內容、地區、經歷、學歷、公司人數、文章編號、工作網址』
11111爬取資料有『日期、工作名稱、公司名稱、公司地址、薪資、工作內容、地區、經歷、學歷、工作網址』

爬取資料並儲存csv格式，使用Pandas處理資料。


## 需安裝的套件
需要安裝的套件：
BeautifulSoup、requests、selenium、pandas、tqdm、openpyxl

## 使用selenium目的
104需要讓網頁加載javascript得到共有多少頁面，使用Headless(無頭)模式在背景執行Chrome，並不會實際看到瀏覽器。

chromedriver下載

https://chromedriver.chromium.org/downloads

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import random
import time
from urllib.parse import quote
from tqdm import tqdm, trange
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pandas as pd
from datetime import date
from IPython.display import clear_output
import os,os.path

# 104-1111人力銀行 爬蟲

In [2]:
def get_todate():
    return date.today()

def selenium_get_Code_104(url):
    chrome_options = Options() # 啟動無頭模式
    chrome_options.add_argument('--headless')  #規避google bug
    chrome_options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(chrome_options=chrome_options)
    driver.get(url)
    save = driver.page_source
    driver.quit()#關閉瀏覽器
    soup = BeautifulSoup(save, "html.parser")
    page = soup.select('.page-select.js-paging-select.gtm-paging-top')[0].find_all('option')[-1].get('value')
    return page

def read_url(url):
    USER_AGENT_LIST = [
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
        "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
        "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
        "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
        "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
        "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
        "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
        "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
        "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
        "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)",
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 LBBROWSER",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
        "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)",
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; 360SE)",
        "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
        "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
        "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
        "Mozilla/5.0 (iPad; U; CPU OS 4_2_1 like Mac OS X; zh-cn) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8C148 Safari/6533.18.5",
        "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b13pre) Gecko/20110307 Firefox/4.0b13pre",
        "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:16.0) Gecko/20100101 Firefox/16.0",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
        "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10",
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
        ]
    USER_AGENT = random.choice(USER_AGENT_LIST)
    headers = {'user-agent': USER_AGENT}
    s = requests.Session()
    req = s.get(url, headers = headers)
    soup = BeautifulSoup(req.text, "html.parser")
    return soup

def csv_column_104(path_csv, key_txt): #建立行標題
    with open(path_csv + '.csv', mode='a+', newline='', encoding='utf-8') as employee_file: 
        employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        employee_writer.writerow(['日期', '工作名稱', '公司名稱', '公司地址', '薪資', '工作內容', '地區', '經歷', '學歷', '公司人數', '文章編號', '工作網址'])

def find_title_104(key_txt):
    #路徑組合
    today = get_todate()
    path_csv = "%s" % os.getcwd() + '\\' + 'jobs_csv\\'+ str(today) + key_txt + '_104人力銀行'
    if not os.path.isdir('jobs_csv'): # 確認是否有jobs_csv資料夾  沒有則返回Ture
        os.mkdir('jobs_csv') # 建立jobs_csv資料夾
        print('建立jobs_csv資料夾完成')
    csv_column_104(path_csv, key_txt) #建立行標題
    csv_save = ""
    key = quote(key_txt)
    #  104 api searchTempExclude=2  -> 設定排除派遣
    find_page_url = 'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword={0}&order=15&asc=0&page=1&mode=s&jobsource=2018indexpoc&searchTempExclude=2'.format(key)
    get_sum_page = int(selenium_get_Code_104(find_page_url))
    print('共有：' + str(get_sum_page) + ' 頁')
    for i in tqdm(range(1, get_sum_page+1)):  #set page 1 to find all max page ,tqdm讀取進度條
        url = 'https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword={0}&order=15&asc=0&page={1}&mode=s&jobsource=2018indexpoc&searchTempExclude=2'.format(key, i) 
        #time.sleep(random.randint(2,10)) #隨機等待
        soup = read_url(url) #讀取網頁
        print('目前爬取頁面是：' + url)
        for title_1 in soup.select('.b-block__left'):
            #有三個資料是無資料的，遇到無資料就跳過這個迴圈
            if title_1.select('.b-list-inline.b-clearfix.job-list-item__company') != soup.select('.b-block__left')[0].select('.b-list-inline.b-clearfix.job-list-item__company'):
                #日期
                try:
                    #正常代表找到 讚 廣告 (業主買廣告)，發生異常代表找不到 讚，執行except找日期 
                    date_match__ = title_1.select('.b-icon--gray.b-icon--w18')[0].select('use')[0]
                    date = '廣告'
                except:
                    date = title_1.select('.b-tit__date')[0].get_text().replace('\n','').replace(' ','')
        
                #地區
                area = title_1.select('.b-list-inline.b-clearfix.job-list-intro.b-content')[0].find('li').get_text()
                #經歷(年資)
                experience = title_1.select('.b-list-inline.b-clearfix.job-list-intro.b-content')[0].find_all('li')[1].get_text()
                try: #業者沒有輸入學歷，遇到錯誤處理
                    #學歷
                    education = title_1.select('.b-list-inline.b-clearfix.job-list-intro.b-content')[0].find_all('li')[2].get_text()
                except:
                    education = ""
                #工作網址
                title_url = title_1.select('.js-job-link')[0].get('href')[2:]
                #get 文章編號
                title_str = title_url.split('?')[0].split('/')[-1] #get 文章編號
                #標題名稱
                title = title_1.select('.js-job-link')[0].get_text() #get title
                #print(title + title_url + area)
                #公司名
                company_name = title_1.select('li')[1].find('a').get('title').split('\n')[0][4:]
                try:
                    #公司地址
                    company_address = title_1.select('li')[1].find('a').get('title').split('\n')[1][5:]
                except:
                    company_address = ""
                try:
                    #簡介
                    introduction = title_1.select('.job-list-item__info.b-clearfix.b-content')[0].get_text()
                    #處理string \r \n5 \n轉成''
                    introduction = introduction.replace('\r','').replace('\n5','').replace('\n','')
                except:
                    introduction = ""
                #薪資
                try:
                    salary = title_1.select('.b-tag--default')[0].get_text()
                except:
                    salary = 0 #沒有寫薪資或待遇面議，設定 0
                    
                if salary == '待遇面議':
                    salary = "待遇面議"
                else: #數字處理 25000~35000 取25000最低薪資為主要，三位數 = 日薪，四位數 = 論件計酬
                    try:
                        salary = re.search('\d+.\d+', salary).group()
                    except:
                        salary = 0
                #員工人數
                try:
                    people = title_1.select('.b-tag--default')[1].get_text()
                except:
                    people = ""
                #clear_output() # 清除輸出 用於清除進度讀，註解#不使用：用來檢查出錯的網址

                with open(path_csv + '.csv', mode='a+', newline='', encoding='utf-8') as employee_file: #w
                    employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    employee_writer.writerow([date, title, company_name, company_address, salary, introduction, area, experience, education, people, title_str, title_url])
            else:
                continue
    return print('爬取104完成：請開啟csv檔案')


# input_go = input('輸入關鍵字')
# save_title_data = find_title_104(input_go)




############ 1111人力銀行 #########


def selenium_get_Code_1111(url):
    chrome_options = Options() # 啟動無頭模式
    chrome_options.add_argument('--headless')  #規避google bug
    chrome_options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(chrome_options=chrome_options)
    driver.get(url)
    save = driver.page_source
    driver.quit()#關閉瀏覽器
    soup = BeautifulSoup(save, "html.parser")
    
    page = soup.select('.custom-select')[0].select('option')[0].text
    page = page.split('/')
    page = page[1].strip(' ')
    return page


def csv_column_1111(path_csv): #建立行標題
    with open(path_csv + '.csv', mode='a+', newline='', encoding='utf-8') as employee_file: 
        employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        employee_writer.writerow(['日期', '工作名稱', '公司名稱', '公司地址', '薪資', '工作內容', '地區', '經歷', '學歷', '工作網址'])

def find_data_1111(soup):
    #錢錢
    mnone = soup.select('.needs') 
    #縣市區域
    location = soup.select('.needs')
    #日期
    get_date = soup.select('.date')
    #簡介
    jbInfoTxt = soup.select('.jbInfoTxt')
    #網址
    jobs_url = soup.select('.position0')
    #公司名稱、類別、住址
    company_data = soup.select('.d-md-flex')
    #工作標題
    title = soup.select('.position0')
    #工作經驗
    jobs_exp = soup.select('.needs')
    # 學歷
    education = soup.select('.needs')
    return mnone, location, jbInfoTxt, jobs_url, company_data, title, jobs_exp, get_date, education

        
def find_title_1111(key_txt):
    #路徑組合
    today = get_todate()
    path_csv = "%s" % os.getcwd() + '\\' + 'jobs_csv\\' + str(today) + key_txt + '_1111人力銀行'
    if not os.path.isdir('jobs_csv'): # 確認是否有jobs_csv資料夾  沒有則返回Ture
        os.mkdir('jobs_csv') # 建立jobs_csv資料夾
        print('建立jobs_csv資料夾完成')
    csv_column_1111(path_csv) #建立行標題
    key = quote(key_txt)
    find_page_url = 'https://www.1111.com.tw/job-bank/job-index.asp?si=1&ss=s&ks={0}&page=1'.format(key)
    #取得最大page數
    get_sum_page = int(selenium_get_Code_1111(find_page_url))
    print('共有：' + str(get_sum_page) + ' 頁')
    
    for i in tqdm(range(1, get_sum_page+1)):
        url = 'https://www.1111.com.tw/job-bank/job-index.asp?si=1&ss=s&ks={0}&page={1}'.format(key, i)
        soup = read_url(url) #讀取網頁
        #讀取網頁資料
        mnone, location, jbInfoTxt, jobs_url, company_data, title, jobs_exp, get_date, education = find_data_1111(soup)
        print('目前爬取頁面是：' + url)
        for mnone, location, jbInfoTxt, jobs_url, company_data, title, jobs_exp, get_date, education in zip(mnone, location, jbInfoTxt, jobs_url, company_data, title, jobs_exp, get_date, education):
            #錢 取最低薪資
            try:
                mnone = mnone.find_all("span")[1].get_text()
                get_mone = re.search('\d+.\d+', mnone).group()
            except:
                get_mone = '面議（經常性薪資4萬/月含以上）' #也可以直接給40000
            #日期
            get_date = get_date.get_text()[5:]

            #縣市區域
            location = location.find_all("span")[0].get_text()

            #簡介
            jbInfoTxt = jbInfoTxt.get_text().replace("\xa0", "") #刪除\xa0

            #工作網址
            jobs_url = 'https://www.1111.com.tw{0}'.format(jobs_url.find('a').get('href'))

            #公司名
            company = company_data.find_all('a')[0].get('title').replace('\r','').split('\n')[0][6:]

            #公司分類 目前暫不使用
            category = company_data.find_all('a')[0].get('title').replace('\r','').split('\n')[1][6:]

            #公司地址
            address = company_data.find_all('a')[0].get('title').replace('\r','').split('\n')[2][6:]

            #工作標題
            title = title.find('a').get('title')

            # 工作經驗
            jobs_exp = jobs_exp.find_all("span")[2].get_text()

            # 學歷
            education = education.find_all("span")[3].get_text()
            # 儲存

            # clear_output() # 清除輸出 用於清除進度讀

            with open(path_csv + '.csv', mode='a+', newline='', encoding='utf-8') as employee_file: #w
                employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                employee_writer.writerow([get_date, title, company, address, get_mone, jbInfoTxt, location, jobs_exp, education, jobs_url])
    return print('爬取1111完成：請開啟csv檔案')


# 執行爬蟲

使用 find_title_104('金融') find_title_1111('金融')

In [3]:
find_key = '金融'
find_title_104(find_key)
find_title_1111(find_key)

C:\Users\love\anaconda3\envs\mytest\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  
  0%|                                                                                          | 0/150 [00:00<?, ?it/s]

共有：150 頁
目前爬取頁面是：https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=%E9%87%91%E8%9E%8D&order=15&asc=0&page=1&mode=s&jobsource=2018indexpoc&searchTempExclude=2


  1%|▌                                                                                 | 1/150 [00:01<03:13,  1.30s/it]

目前爬取頁面是：https://www.104.com.tw/jobs/search/?ro=0&kwop=7&keyword=%E9%87%91%E8%9E%8D&order=15&asc=0&page=2&mode=s&jobsource=2018indexpoc&searchTempExclude=2


  1%|█                                                                                 | 2/150 [00:02<02:51,  1.16s/it]

...

目前爬取頁面是：https://www.1111.com.tw/job-bank/job-index.asp?si=1&ss=s&ks=%E9%87%91%E8%9E%8D&page=146


 98%|██████████████████████████████████████████████████████████████████████████████▍ | 147/150 [01:51<00:02,  1.04it/s]

目前爬取頁面是：https://www.1111.com.tw/job-bank/job-index.asp?si=1&ss=s&ks=%E9%87%91%E8%9E%8D&page=147


100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [01:53<00:00,  1.32it/s]

目前爬取頁面是：https://www.1111.com.tw/job-bank/job-index.asp?si=1&ss=s&ks=%E9%87%91%E8%9E%8D&page=148
目前爬取頁面是：https://www.1111.com.tw/job-bank/job-index.asp?si=1&ss=s&ks=%E9%87%91%E8%9E%8D&page=149
目前爬取頁面是：https://www.1111.com.tw/job-bank/job-index.asp?si=1&ss=s&ks=%E9%87%91%E8%9E%8D&page=150
爬取1111完成：請開啟csv檔案


# Pandas 過濾工作

In [4]:
import glob
glob.glob('jobs_csv/*.csv')

['jobs_csv\\2020-06-05金融_104人力銀行.csv',
 'jobs_csv\\2020-06-05金融_1111人力銀行.csv',
 'jobs_csv\\2020-07-06金融_104人力銀行.csv',
 'jobs_csv\\2020-07-06金融_1111人力銀行.csv']

In [5]:
#讀取爬取到的資料
get_file_name = glob.glob('jobs_csv/*.csv')
data_104 = pd.read_csv(get_file_name[2])
data_1111 = pd.read_csv(get_file_name[3])

查看資料

In [6]:
data_1111.head()

,日期,工作名稱,公司名稱,公司地址,薪資,工作內容,地區,經歷,學歷,工作網址
0,2020-06-23,電話行銷客服人員-保障底薪+獎金+供餐,全球華人股份有限公司(1111人力銀行),台北市松山區南京東路,"25,000",1.內勤電話開發潛在客戶，拓展市場，以達成業績目標。 2.負責業務推展接洽及訂單處理。 3....,台北市松山區,1年工作經驗以上,"高中職,專科,大學,碩士,博士",https://www.1111.com.tw/job/85973247/
1,2020-06-09,教育訓練師,全球華人股份有限公司高雄分公司(1111人力銀行高雄分公司),高雄市前金區中山二路,"30,000",1.負責新進員工電話行銷教育訓練。2.電話行銷課程規劃執行、教材製作、學習績效評估。3.新進...,高雄市前金區,1年工作經驗以上,"專科,大學",https://www.1111.com.tw/job/86023657/
2,2020-06-08,電銷客服專員【週休二日】,全球華人股份有限公司高雄分公司(1111人力銀行高雄分公司),高雄市前金區中山二路,"25,000",你喜歡講話嗎?快加入我們你喜歡賺錢嗎?快加入我們你喜歡廣結善緣嗎?快加入我們你喜歡學習成長嗎...,高雄市前金區,經驗不拘,"高中職,專科,大學,碩士",https://www.1111.com.tw/job/86023633/
3,2020-07-02,會計記帳人員,仕展記帳士事務所,桃園市中壢區寧夏二街,"25,000",1.在會計原則之下，從事辦理會計、財務處理記錄等工作。2.代客記帳及稅務申報業務3.協助各項...,桃園市中壢區,經驗不拘,"專科,大學,碩士",https://www.1111.com.tw/job/85204068/
4,2020-07-06,會計助理/專員,金盟營造有限公司,台北市中山區明水路,"26,000",1. 收發、記錄及彙整憑證，登錄至會計軟體。2. 審核發票、單據及一般會計帳務處理。3. 處...,台北市中山區,1年工作經驗以上,"大學,碩士,博士",https://www.1111.com.tw/job/85985749/


In [7]:
data_104.head()

,日期,工作名稱,公司名稱,公司地址,薪資,工作內容,地區,經歷,學歷,公司人數,文章編號,工作網址
0,廣告,富邦行銷/富昇保代-電話行銷專員(台南/高雄),富邦行銷股份有限公司,台南市中西區民生路二段279號5樓,"25,000",1.屬全職辦公室內勤、正職人員編制。 2.專責電話線上商品行銷與服務之工作，不出差外訪，無需...,台南市中西區,經歷不拘,高中,員工400人,56so8,www.104.com.tw/job/56so8?jobsource=hotjob_chr
1,廣告,行銷專員CA(正職),富邦人壽保險股份有限公司-中辰通訊處(核准文號109A01號),台中市西屯區市政路386號26樓之3中辰通訊處,158,歡迎尋求成長機會，肯吃苦、肯努力、挑戰高薪的您加入。1.銷售人身保險商品、客戶經營服務並提供...,台中市西屯區,經歷不拘,高中,員工27825人,5aifk,www.104.com.tw/job/5aifk?jobsource=hotjob_chr
2,廣告,Branch Universal Sales_全方位金融服務專員_台北(無經驗可_完整培訓_...,花旗(台灣) 商業銀行股份有限公司,台北市中山區德惠街9號,"26,000",1.透過駐點花旗分行開發客戶，提供信用卡 / 信用貸款 / 銀行帳戶，針對客戶需求推薦最適當...,台北市中山區,經歷不拘,專科,外商公司,3w4bi,www.104.com.tw/job/3w4bi?jobsource=hotjob_chr
3,廣告,風險控制管理師 Risk Management,智鈦星平台服務有限公司,台北市信義區市民大道六段288號8樓之8,"35,000",工作內容：1. 監控日常狀態，並以數據資料產出相關報告管控風險。2. 辨視異常狀況及高風險使...,台北市信義區,1年以上,專科,員工70人,6wyuy,www.104.com.tw/job/6wyuy?jobsource=hotjob_chr
4,廣告,新光投信-海外基金經理人,新光金融控股股份有限公司(新光金控/新光人壽/新光銀行/新光投信),台北市中山區南京東路二段123號12樓,待遇面議,1. 海外基金投資管理2. 海外產業、市場研究及個股研究分析3. 協助基金銷售、客戶簡報,台北市中山區,2年以上,碩士,上市上櫃,6spmi,www.104.com.tw/job/6spmi?jobsource=hotjob_chr


## 選擇想要的資料

日期、工作名稱、公司地址、薪資、工作內容、地區、經歷、學歷、工作網址
依想要的順序排序

In [8]:
new_data_1111 =  data_1111[['日期', '工作名稱', '薪資', '地區', '工作內容', '公司地址', '經歷', '學歷', '工作網址']]
new_data_104 = data_104[['日期', '工作名稱', '薪資', '地區', '工作內容', '公司地址', '經歷', '學歷', '工作網址']]

## 需要接資料，方便一次過濾資料及處理資料

In [9]:
#檢查shape 
print('104：' + str(new_data_104.shape))
print('1111：' + str(new_data_1111.shape))


104：(2950, 9)
1111：(1705, 9)


## 接資料

In [10]:
concat_104_1111 = pd.concat([new_data_1111, new_data_104], axis=0)

In [11]:
#row數量
concat_104_1111.shape


(4655, 9)

index重設

In [12]:
concat_104_1111.tail() #資料最後面的index 不是5952，需要重新建立index

,日期,工作名稱,薪資,地區,工作內容,公司地址,經歷,學歷,工作網址
2945,11/22,消金業務分行駐點人員(高雄),"34,000",高雄市左營區,駐點於分行，負責服務消金業務(房貸、信用卡、房貸壽險、產險…等)，提供客戶消金商品諮詢與規劃...,高雄市左營區(工作地點：高雄地區，依公司規定分派單位),1年以上,專科,www.104.com.tw/job/57y40?jobsource=2018indexpoc
2946,6/19,會計,"35,000",台北市信義區,1. 客戶應收帳務和逾期帳管催收管理 2. 收入帳務處理及發票開立 3. 營業稅調節表編制及...,台北市信義區松仁路100號30樓,3年以上,專科,www.104.com.tw/job/67iug?jobsource=2018indexpoc
2947,7/05,銀行理專,待遇面議,新北市中和區,\t1.提供有關證券、債券、市場條件、投資前途及財務狀況等擬投資之資料。 2.分析證券、債券...,新北市中和區連城路,經歷不拘,專科,www.104.com.tw/job/6ro5l?jobsource=2018indexpoc
2948,6/23,儲備處經理培育專案(承攬性質，非勞動契約)(核准編號X1Q-109-001),1~9,台北市中正區,儲備處經理專案優勢：「儲備處經理培育專案」是新光人壽專銷制業務單位全新推出之儲備幹部培訓專案...,台北市中正區忠孝西路一段66號17樓,經歷不拘,大學,www.104.com.tw/job/6z5rm?jobsource=2018indexpoc
2949,6/29,會計,"28,000",新北市林口區,1.各項費用支付之帳務處理。2.廠商貨款或費用等應付款項帳務。3.客戶應收款項帳務。4.其他...,新北市林口區中山路81巷1號1樓,2年以上,專科,www.104.com.tw/job/3s95d?jobsource=2018indexpoc


In [13]:
concat_104_1111.reset_index(drop=True, inplace=True) #drop=True 刪除原有index , inplace 直接更新現有的DataFrame

In [14]:
concat_104_1111.tail()

,日期,工作名稱,薪資,地區,工作內容,公司地址,經歷,學歷,工作網址
4650,11/22,消金業務分行駐點人員(高雄),"34,000",高雄市左營區,駐點於分行，負責服務消金業務(房貸、信用卡、房貸壽險、產險…等)，提供客戶消金商品諮詢與規劃...,高雄市左營區(工作地點：高雄地區，依公司規定分派單位),1年以上,專科,www.104.com.tw/job/57y40?jobsource=2018indexpoc
4651,6/19,會計,"35,000",台北市信義區,1. 客戶應收帳務和逾期帳管催收管理 2. 收入帳務處理及發票開立 3. 營業稅調節表編制及...,台北市信義區松仁路100號30樓,3年以上,專科,www.104.com.tw/job/67iug?jobsource=2018indexpoc
4652,7/05,銀行理專,待遇面議,新北市中和區,\t1.提供有關證券、債券、市場條件、投資前途及財務狀況等擬投資之資料。 2.分析證券、債券...,新北市中和區連城路,經歷不拘,專科,www.104.com.tw/job/6ro5l?jobsource=2018indexpoc
4653,6/23,儲備處經理培育專案(承攬性質，非勞動契約)(核准編號X1Q-109-001),1~9,台北市中正區,儲備處經理專案優勢：「儲備處經理培育專案」是新光人壽專銷制業務單位全新推出之儲備幹部培訓專案...,台北市中正區忠孝西路一段66號17樓,經歷不拘,大學,www.104.com.tw/job/6z5rm?jobsource=2018indexpoc
4654,6/29,會計,"28,000",新北市林口區,1.各項費用支付之帳務處理。2.廠商貨款或費用等應付款項帳務。3.客戶應收款項帳務。4.其他...,新北市林口區中山路81巷1號1樓,2年以上,專科,www.104.com.tw/job/3s95d?jobsource=2018indexpoc


## 清理資料

清理 薪資的資料：
    1. 將 待遇面議 轉換成 0
    2. 將 面議（經常性薪資4萬/月含以上）轉成 40000
    3. 將 時薪 例如：150~200 轉成只取最低時薪 150
    4. 移除數字中的, 例如"30,000"將,移除

In [15]:
#將四萬改成0  但是也可以改成40,000
concat_104_1111['薪資'] = concat_104_1111.薪資.str.replace('面議（經常性薪資4萬/月含以上）', '40000')

#將待遇面議改成0
concat_104_1111['薪資'] = concat_104_1111.薪資.str.replace('待遇面議', '0')

#時薪 取最低時薪
concat_104_1111.loc[concat_104_1111.薪資.str.contains('~'),'薪資'] = concat_104_1111.loc[concat_104_1111.薪資.str.contains('~'),'薪資'].str.split('~').str[0]

#數字有"30,000"將,移除
concat_104_1111.loc[:,'薪資'] = concat_104_1111['薪資'].apply(lambda x:x.replace(',','')) 

# 薪資轉成int type
# concat_104_1111.loc[:,'薪資'] = concat_104_1111['薪資'].astype(int)
concat_104_1111.loc[:,'薪資'] = pd.to_numeric(concat_104_1111['薪資'])

In [16]:
#薪資 已轉成 int64
concat_104_1111.dtypes

日期      object
工作名稱    object
薪資       int64
地區      object
工作內容    object
公司地址    object
經歷      object
學歷      object
工作網址    object
dtype: object

## 過濾

找想要的工作

過濾方法：可以參考這篇 https://ithelp.ithome.com.tw/articles/10194003

In [30]:
#1萬至3萬之間、地區：台北市 這種過濾方法會寫越寫長
#concat_104_1111.loc[concat_104_1111.薪資 > 10000].loc[concat_104_1111.薪資 < 30000].loc[concat_104_1111.地區.str.contains('台北市')]


#3萬以上
mask1 = concat_104_1111.薪資 >= 30000
mask2 = concat_104_1111.地區.str.contains('台北市')
mask3 = concat_104_1111.地區.str.contains('新北市')
mask4 = concat_104_1111.地區.str.contains('桃園市')
mask5 = concat_104_1111.工作名稱.str.contains('分析')
mask6 = concat_104_1111.工作內容.str.contains('證券') #工作內容簡介也許會有相關的工作
#mask7 =  

#搜尋 3萬以上 工作名稱有"證卷"關鍵字、區域 台北市 新北市
# & = and , | = or   
concat_104_1111.loc[((mask2 | mask3 | mask4) & mask1 & mask5) | ((mask2 | mask3 | mask4) & mask1 & mask6)].head()


,日期,工作名稱,薪資,地區,工作內容,公司地址,經歷,學歷,工作網址
19,2020-07-03,S-金融理財顧問Financial Advisor(板盛分公司),40000,新北市板橋區,優先考量：具有客戶信任基礎為佳，提供高底薪高獎金制度。1. FA的特質:正面、積極、可影響他...,台北市中山區八德路,2年工作經驗以上,"大學,碩士,博士",https://www.1111.com.tw/job/78592271/
22,2020-07-03,S-金融理財顧問-Financial Advisor(敦南分公司),40000,台北市大安區,1. 負責證券、期貨、選擇權之開發及接單。2. 開發財富管理客戶.協銷各項金融商品（如：基金...,台北市中山區八德路,經驗不拘,"專科,大學,碩士,博士",https://www.1111.com.tw/job/78513979/
50,2020-06-29,科技金融_私人銀行系統分析師,40000,台北市南港區,1. 需求分析、系統設計及系統文件撰寫。2. 新系統專案建置。3. 系統開發維運及問題排除。,台北市南港區經貿二路,3年工作經驗以上,"大學,碩士,博士",https://www.1111.com.tw/job/91510390/
57,2020-07-05,S-理財專員(板新分公司),30000,新北市板橋區,1. 提供有關證券、債券、市場條件、投資前途及財務狀況等擬投資之資料。2. 分析證券、債券、...,台北市中山區八德路,經驗不拘,"高中職,專科,大學,碩士,博士",https://www.1111.com.tw/job/86012506/
64,2020-07-04,桃園-理財業務人員,32576,桃園市,【學歷及資格條件】1.國內外大專以上學歷畢業者(具理財業務經驗3年以上者除外)。2.具備保險...,高雄市左營區博愛二路,經驗不拘,不拘,https://www.1111.com.tw/job/79036526/


In [27]:
str(get_todate()) + '金融'

'2020-06-05金融'

In [22]:
#儲存成 Excel格式檔
file_name = str(get_todate()) + '金融' #今天日期+檔案名稱
save_excel = concat_104_1111.loc[((mask2 | mask3 | mask4) & mask1 & mask5) | ((mask2 | mask3 | mask4) & mask1 & mask6)].head()
save_excel.to_excel('jobs_csv/{}.xlsx'.format(file_name), sheet_name='passengers', index=False)

In [30]:
#我想兼差打工 50至1000之間
concat_104_1111.loc[concat_104_1111.薪資 > 50].loc[concat_104_1111.薪資 < 1000]

,日期,工作名稱,薪資,地區,工作內容,公司地址,經歷,學歷,工作網址
224,2020-06-01,B-《2020暑期實習計畫-金融市場處》金融市場專案研究,160,台北市中正區,《實習內容》• 每日上午: 參加晨會討論與研讀市場資訊；在(交易員)旁觀察市場動態/交易決策...,台北市中山區八德路,經驗不拘,大學,https://www.1111.com.tw/job/91499883/
229,2020-06-01,B-《2020暑期實習計畫-金融市場處》金融市場專案研究,160,台北市中正區,《實習內容》• 每日上午: 參加晨會討論與研讀市場資訊；在(交易員)旁觀察市場動態/交易決策...,台北市中山區八德路,經驗不拘,大學,https://www.1111.com.tw/job/91499883/
235,2020-06-01,B-《2020暑期實習計畫-法人金融處》法人金融實習人員,160,台北市中正區,1. FI期中覆審工作2. 法金CRM(客戶關係管理)3. 電廠資料運用案：過往資料建檔、資...,台北市中山區八德路,經驗不拘,碩士,https://www.1111.com.tw/job/91499504/
360,2020-06-01,B-《2020暑期實習計畫-電子金融處》數位金融產品專案人員,160,台北市中正區,• 產品設計1. 參與產品功能設計發想、規劃與討論2. 基本網頁排版及設計3. 數位金融產品...,台北市中山區八德路,經驗不拘,碩士,https://www.1111.com.tw/job/91499768/
409,2020-06-03,[兼職] 財經/金融_英翻中翻譯師,100,台北市大安區,工作描述:1. 依據Localization標準，翻譯財經/金融相關類型文件(英翻中)。2....,台北市大安區信義路,經驗不拘,"專科,大學,碩士,博士",https://www.1111.com.tw/job/91473326/
...,...,...,...,...,...,...,...,...,...
4490,5/20,財務部工讀生(6 months),160,台北市士林區,歡迎財務或會計相關科系的學生們能透過在美商500大公司的財務部實習機會讓自己所學與實務做一結...,台北市士林區承德路四段172號10樓,經歷不拘,專科,www.104.com.tw/job/641hv?jobsource=2018indexpoc
4507,12/12,F0006_會計工讀生(長期),158,台北市中正區,TutorABC屢獲世界級投資者的關注，分別於2015年10月份獲得近2億美元C輪投資，以及...,台北市中正區羅斯福路二段100號20樓（古亭9號出口）,經歷不拘,專科,www.104.com.tw/job/3xpha?jobsource=2018indexpoc
4626,6/01,財務部-會計工讀生,158,台北市中山區,1. 負責一般帳務處理(包含傳票編製及單據審核作業等)2. 傳票整理及歸檔3. 協助庶務性行...,台北市中山區敬業四路33號,經歷不拘,高中,www.104.com.tw/job/6uc8z?jobsource=2018indexpoc
4668,5/14,工研院巨資中心_產業資訊彙整人員(M0),170,新竹縣竹東鎮,常態性收集與整理相關技術研發與產業動向，並定期產出整理報告供計畫主管與同仁參考。工作內容包含...,新竹縣竹東鎮,經歷不拘,大學,www.104.com.tw/job/6xyl0?jobsource=2018indexpoc
